In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import time
import timeit
%matplotlib inline

# Algorithm Complexity Demo

## Covariance Matrix
(from https://en.wikipedia.org/wiki/Covariance_matrix)

In probability theory and statistics, a covariance matrix (also known as dispersion matrix or variance–covariance matrix) is a matrix whose element in the i, j position is the covariance between the i-th and j-th elements of a random vector. A random vector is a random variable with multiple dimensions. Each element of the vector is a scalar random variable. Each element has either a finite number of observed empirical values or a finite or infinite number of potential values. The potential values are specified by a theoretical joint probability distribution.

Intuitively, the covariance matrix generalizes the notion of variance to multiple dimensions. As an example, the variation in a collection of random points in two-dimensional space cannot be characterized fully by a single number, nor would the variances in the x and y directions contain all of the necessary information; a 2×2 matrix would be necessary to fully characterize the two-dimensional variation.

Because the covariance of the i-th random variable with itself is simply that random variable's variance, each element on the principal diagonal of the covariance matrix is the variance of one of the random variables. Because the covariance of the i-th random variable with the j-th one is the same thing as the covariance of the j-th random variable with the i-th one, every covariance matrix is symmetric. In addition, every covariance matrix is positive semi-definite.

### Variance
(from http://stattrek.com/matrix-algebra/covariance-matrix.aspx)

Variance is a measure of the variability or spread in a set of data. Mathematically, it is the average squared deviation from the mean score. We use the following formula to compute variance: 
![image](Algorithms/VarEQ.png) 


where
```
N is the number of scores in a set of scores
X(bar) is the mean of the N scores.
xi is the ith raw score in the set of scores
```


In [ ]:
def mean_naive(X):
    """Compute the mean for a dataset by iterating over the dataset
    
    Arguments
    ---------
    X: (N, D) ndarray representing the dataset.
    
    Returns
    -------
    mean: (D, ) ndarray which is the mean of the dataset.
    """
    N, D = X.shape
    mean = np.zeros(D)
    
    for d in range(D):
        for n in range(N):
            mean[d] += X[n,d]* (1/(N))
    
    
    return mean




### Covariance

Covariance is a measure of the extent to which corresponding elements from two sets of ordered data move in the same direction. We use the following formula to compute covariance.
![image](Algorithms/CovEQ.png)

where
```
N is the number of scores in each set of data
A(bar) is the mean of the N scores in the first data set
ai is the ithe raw score in the first set of scores
B(bar) is the mean of the N scores in the second data set
bi is the ithe raw score in the second set of scores
```

### Covariance Matrix

Variances and covariances are often displayed together in a matrix. The variances appear along the diagonal and covariances appear in the off-diagonal elements, as shown below.
![image](Algorithms/CovMatrixEQ2.png)

In [ ]:
def cov_naive(X):
    """Compute the covariance for a dataset
    Arguments
    ---------
    X: (N, D) ndarray representing the dataset.
    
    Returns
    -------
    covariance: (D, D) ndarray which is the covariance matrix of the dataset.
    
    """
    N, D = X.shape
    covariance = np.zeros((D, D))
    
    column_means = mean_naive(X)
    
    numerator = 0
    
    for row in range(D):
        
        for col in range(D):
            
            for n in range(N):
                numerator += (X[n,row] - column_means[row]) * (X[n,col] - column_means[col])
            
            covariance[row, col] = numerator * (1/N)
            numerator = 0
    
    return covariance

In [ ]:
#Let's use built-in functions in numpy to get the mean an covariance matrix instead

def mean(X):
    mean = X.mean(axis=0)
    return mean

def cov(X):
    covariance_matrix = np.cov(np.transpose(X))
    
    return covariance_matrix

In [ ]:
# We have some huge data matrix, and we want to compute its mean
X = np.random.randn(100000, 20)

In [ ]:
# Benchmarking time for computing mean
%time mean_naive(X)
%time mean(X)
pass

In [ ]:
# Benchmarking time for computing covariance
%time cov_naive(X)
%time cov(X)
pass

In [ ]:
def time(f, repeat=100):
    """A helper function to time the execution of a function.
    
    Arguments
    ---------
    f: a function which we want to time it.
    repeat: the number of times we want to execute `f`
    
    Returns
    -------
    the mean and standard deviation of the execution.
    """
    times = []
    for _ in range(repeat):
        start = timeit.default_timer()
        f()
        stop = timeit.default_timer()
        times.append(stop-start)
    return np.mean(times), np.std(times)

In [ ]:
#THIS WILL TAKE ~1 min depending on your computer
fast_time = []
slow_time = []

for size in np.arange(100, 5000, step=100):
    X = np.random.randn(size, 20)
    f = lambda : mean(X)
    mu, sigma = time(f)
    fast_time.append((size, mu, sigma))
    
    f = lambda : mean_naive(X)
    mu, sigma = time(f)
    slow_time.append((size, mu, sigma))

fast_time = np.array(fast_time)
slow_time = np.array(slow_time)


fig, ax = plt.subplots()
ax.errorbar(fast_time[:,0], fast_time[:,1], fast_time[:,2], label='fast mean', linewidth=2)
ax.errorbar(slow_time[:,0], slow_time[:,1], slow_time[:,2], label='naive mean', linewidth=2)
ax.set_xlabel('size of dataset')
ax.set_ylabel('running time')
plt.legend();

---

![image](Algorithms/RuntimeMean.png)

# DONT RUN THE NEXT 2 CELLS

In [ ]:
#THIS MIGHT BE OVER AN HOUR depending on your computer
fast_time_cov = []
slow_time_cov = []
for size in np.arange(100, 5000, step=100):
    X = np.random.randn(size, 20)
    f = lambda : cov(X)               
    mu, sigma = time(f) 
    fast_time_cov.append((size, mu, sigma))
    
    f = lambda : cov_naive(X)         
    mu, sigma = time(f) 
    slow_time_cov.append((size, mu, sigma))

fast_time_cov = np.array(fast_time_cov)
slow_time_cov = np.array(slow_time_cov)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(fast_time_cov[:,0], fast_time_cov[:,1], fast_time_cov[:,2], label='fast covariance', linewidth=2)
ax.errorbar(slow_time_cov[:,0], slow_time_cov[:,1], slow_time_cov[:,2], label='naive covariance', linewidth=2)
ax.set_xlabel('size of dataset')
ax.set_ylabel('running time')
plt.legend();


![image](Algorithms/RuntimeCov.png)